In [ ]:
import praw
import pytz
import pandas as pd
from datetime import datetime

client_id='rE_BWSQJPWyu5W7tP1keYw'
client_secret='Cd9uxRBwh5LnNsfxWdyGGIJJAX9Z9g'
user_agent='windows:aPigeon:1.0 (by /u/AP1g3on)'

# 创建 Reddit 实例
reddit = praw.Reddit(
    client_id=client_id,
    client_secret=client_secret,
    user_agent=user_agent
)

In [ ]:
subreddit = reddit.subreddit('EufyCam')
posts_list = []

for submission in subreddit.new(limit=None):
    post_dict = {
        'Title': submission.title,
        'Score': submission.score,
        'Post_ID': submission.id,
        'URL': submission.url,
        'Text': submission.selftext,
        'Comments': submission.num_comments,
        'Timestamp': datetime.fromtimestamp(submission.created_utc, tz=pytz.UTC).strftime('%Y-%m-%d %H:%M:%S')
    }
    posts_list.append(post_dict)

df_posts = pd.DataFrame(columns=['Title', 'Score', 'Post_ID', 'URL', 'Text', 'Comments', 'Timestamp'])
df_posts = pd.DataFrame(posts_list)
df_posts['Text'] = df_posts['Text'].apply(lambda x: str(x).replace('\n', ' '))

post_id_list = df_posts['Post_ID'].tolist()
print(post_id_list)

In [ ]:
# 递归函数来处理评论和子评论
def process_comment(comment, parent_id=None, comments_list=None, post_id=None):
    if comments_list is None:
        comments_list = []

    dict = {
        'Post_ID': post_id,
        'Comment_ID': comment.id,
        'Parent_ID': parent_id,
        'Author': comment.author.name if comment.author else 'Deleted',
        'comment-Body': comment.body,
        'Score': comment.score,
        'Created_UTC': comment.created_utc
    }
    comments_list.append(dict)

    # 处理子评论
    for reply in comment.replies:
        process_comment(reply, parent_id=comment.id, comments_list=comments_list, post_id=post_id)

    return comments_list

comments_list = []
for id in post_id_list:
    submission = reddit.submission(id=id)
    submission.comments.replace_more(limit=0)  # 移除 "MoreComments" 对象
    # 处理没有评论的情况
    if submission.num_comments == 0:
            comments_list.append({'Post_ID':id})
    # 遍历顶级评论
    for top_level_comment in submission.comments: 
        comments_list.extend(process_comment(top_level_comment,post_id=id))


columns = ['Post_ID', 'Comment_ID', 'Parent_ID', 'Author', 'comment-Body', 'Score', 'Created_UTC']
df_comments = pd.DataFrame(columns=columns)
df_comments = pd.DataFrame(comments_list)
df_comments['comment-Body'] = df_comments['comment-Body'].apply(lambda x: str(x).replace('\n', ' '))

In [ ]:
# 合并 DataFrame
merged_df = pd.merge(df_posts, df_comments, on='Post_ID', how='inner')
merged_df.to_csv('all.csv', index=False)